# Create SHETRAN Land Cover and Lake Data
*Ben Smith | 12/12/2025*

This script is designed to take online downloads and reconfigure them into raster layers that can be used to setup SHETRAN models.

Todo:
- Change Saltmarsh from Bareground to Shrub and recreate Land Use map.

### Preamble

In [ ]:
import os
import zipfile
import shutil

import rasterio
import numpy as np

import pandas as pd

from rasterio.merge import merge

from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.transform import from_bounds
from rasterio.features import rasterize

import geopandas as gpd

import Functions_Model_Set_Up as she

root = 'I:/SHETRAN_GB_2021/02_Input_Data/01 - National Data Inputs for SHETRAN UK/'
resolution_output = 200

# Create a function for simplifying map and table data by removing duplicates:
def remove_map_df_duplicates(map_path, table_path, ID_col, duplicate_columns, output_suffix='_unique', data_format: str = '%1.1f'):
    """
    Function to remove duplicate entries in a raster and a linked dataframe.
    The duplicates are identified based on the columns in duplicate_columns, and the minimum Raster_ID is used for each group.
    ID_col: string of ID column. e.g. 'Raster_ID'
    duplicate_columns = list of column srings. e.g. ['Flow Mechanism', 'Summary']
    """

    # Read in the table and map:
    print('Reading in data...')
    table = pd.read_csv(table_path)
    map, mc, mr, mx, my, mcs, mnd, _, _ = she.read_ascii_raster(map_path, data_type=int, replace_NA=False)

    # Group using the desired columns and return the raster IDs in each group:
    print('Grouping duplicated data...')
    groups = table.groupby(duplicate_columns)[ID_col].apply(list).reset_index().Raster_ID

    # -- Now run through each group, finding the minimum Raster_ID and changing all other IDs to that in the map.

    # Run throug the groups:
    print('Running through duplicates...')
    for group in groups:
        # Find minimum ID: 
        new_ID = min(group)

        # Change the duplicated IDs to the new ID:
        for old_ID in group:
            ## Table
            # table.loc[table[ID_col] == old_ID, ID_col] = new_ID
            # Map
            map[map == old_ID] = new_ID

    # Drop duplicates from the tbale - the method above will change the IDs but will not remove duplicate rows:
    print('Dropping duplicates...')
    table.drop_duplicates(subset=duplicate_columns, keep='first', inplace=True)#.reset_index(drop=True)

    # Reset the indexes so that they run consecutively:
    print('resetting indexes...')
    counter = 1
    for old_ID in sorted(table[ID_col]):
        # Table:
        table.loc[table[ID_col] == old_ID, ID_col] = counter
        # Map
        map[map == old_ID] = counter
        counter+=1

    # Write out the map:
    print('Writing data...')
    she.write_ascii(array=map, ascii_ouput_path=map_path.replace('.asc', f'{output_suffix}.asc'),
        xllcorner=mx, yllcorner=my, cellsize=mcs, ncols=mc, nrows=mr, NODATA_value=mnd, data_format=data_format)

    # Remove the duplicated rows from the table and write it to csv:
    table.to_csv(table_path.replace('.csv', f'{output_suffix}.csv'), index=False)

## Land Cover Datasets

These are available as 25m and 1km rasters or as vector layers. Vectors are prefered as these allow for greater precision when building lower resolution rasters however these took an unfeasibly long time to process and so rasters were used instead. On inspection these look fully fit for purpose.

All data is CEH Land Cover data (2007), available online for GB and NI (separately):
_https://catalogue.ceh.ac.uk/documents/e02b4228-fdcf-4ab7-8d9d-d3a16441e23d_

The NI data was converted from ING to BNG and then merged with the GB data. The CEH Land use classes were then reclassified to the SHETRAN classes and the data resampled to the required resolution and written as a .asc file in the same format as the other SHETRAN inputs.

| 	**LCM2007 Class**	       | 	**LCM2007 Class Number**	 | 	**SHETRAN Class**	 | 	**SHETRAN Class Number**	 |
|---------------------------|----------------------------|---------------------|----------------------------|
| 	Broadleaved woodland	    | 	1	                        | 	DeciduousForest	   | 	4	                        |
| 	Coniferous Woodland	     | 	2	                        | 	EvergreenForest	   | 	5	                        |
| 	Arable and Horticulture	 | 	3	                        | 	Arable	            | 	1	                        |
| 	Improved Grassland	      | 	4	                        | 	Grass	             | 	3	                        |
| 	Rough grassland	         | 	5	                        | 	Grass	             | 	3	                        |
| 	Neutral Grassland	       | 	6	                        | 	Grass	             | 	3	                        |
| 	Calcareous Grassland	    | 	7	                        | 	Grass	             | 	3	                        |
| 	Acid Grassland	          | 	8	                        | 	Grass	             | 	3	                        |
| 	Fen, Marsh and Swamp	    | 	9	                        | 	Shrub	             | 	6	                        |
| 	Heather	                 | 	10	                       | 	Shrub	             | 	6	                        |
| 	Heather grassland	       | 	11	                       | 	Shrub	             | 	6	                        |
| 	Bog	                     | 	12	                       | 	Shrub	             | 	6	                        |
| 	Montane Habitats	        | 	13	                       | 	Shrub	             | 	6	                        |
| 	Inland Rock	             | 	14	                       | 	BareGround	        | 	2	                        |
| 	Saltwater	               | 	15	                       | 	Water	             | 	8	                        |
| 	Freshwater	              | 	16	                       | 	Water	             | 	8	                        |
| 	Supra-littoral Rock	     | 	17	                       | 	BareGround	        | 	2	                        |
| 	Supra-littoral Sediment	 | 	18	                       | 	BareGround	        | 	2	                        |
| 	Littoral Rock	           | 	19	                       | 	BareGround	        | 	2	                        |
| 	Littoral sediment	       | 	20	                       | 	BareGround	        | 	2	                        |
| 	Saltmarsh                | 21	                        | Shrub               | 6 |
| 	Urban	                   | 	22	                       | 	Urban	             | 	7	                        |
| 	Suburban	                | 	23	                       | 	Urban	             | 	7	                        |

**Step 1 - Reproject the NI data**

This reprojection of the NI data from ING to BNG does affect it slightly, but that shouldn't make much difference in the models.

In [ ]:
# >>> Reproject NI data to BNG - Run once! <<<
# Define input and output file paths
NI_LCM = root + "/Land Use Inputs/LCM 2007 25m Raster/data/LCM2007_NI_25M.tif"
NI_LCM_BNG = root + "/Land Use Inputs/LCM 2007 25m Raster/data/LCM2007_NI_25M_BNG.tif"

# Reproject raster
with rasterio.open(NI_LCM) as src:
    # Define target CRS (British National Grid)
    dst_crs = "EPSG:27700"

    # Calculate transform and dimensions for the target CRS
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds
    )
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    # Write the reprojected raster
    with rasterio.open(NI_LCM_BNG, 'w', **kwargs) as dst:
        reproject(
            source=rasterio.band(src, 1),  # Source data
            destination=rasterio.band(dst, 1),  # Destination array
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.nearest  # Adjust resampling method if needed
        )

**Step 2 - Merge, reclassify, resample to the desired resolution and write**

Change the resolution at the top of this file and rerun for whatever resolutions you need. This seems to be slower for the larger resolutions.

In [21]:
# Reclassification mapping
reclass_mapping = {0: -9999,
    1: 4, 2: 5, 3: 1, 4: 3, 5: 3, 6: 3, 7: 3, 8: 3, 9: 6,
    10: 6, 11: 6, 12: 6, 13: 6, 14: 2, 15: 8, 16: 8,
    17: 2, 18: 2, 19: 2, 20: 2, 21: 2, 22: 7, 23: 7
}

# Paths for your rasters
raster_GB_LCM = root + "/Land Use Inputs/LCM 2007 25m Raster/data/lcm2007gb25m.tif"
raster_NI_LCM = root + "/Land Use Inputs/LCM 2007 25m Raster/data/LCM2007_NI_25M_BNG.tif"

# Open LCM GB and NI raster files:
print('READING')
rasters = [rasterio.open(f) for f in [raster_GB_LCM, raster_NI_LCM]]

# Merge the rasters into a single UK raster:
merged_raster, merged_transform = merge(rasters)

# Create an empty array to hold the reclassified data:
reclassified_data = np.empty(merged_raster.shape)  # np.copy(merged_raster)

# Reclassify from the LCM classes the SHETRAN classes:
for original_value, new_value in reclass_mapping.items():
    reclassified_data[merged_raster == original_value] = new_value

# Change -9999s into nan values so that they do not influence processing:
reclassified_data[reclassified_data == -9999] = np.nan

# Set up an empty array to hole the resampled data:
xmin, ymin, xmax, ymax = 0, 0, 661000, 1241000  # resolution of the existing SHETRAN inputs
new_transform = from_bounds(xmin, ymin, xmax, ymax,
                            width=(xmax - xmin) // resolution_output,
                            height=(ymax - ymin) // resolution_output)
new_shape = ((ymax - ymin) // resolution_output, (xmax - xmin) // resolution_output)
resampled_raster = np.empty(new_shape)

# Resample the data to the desired resolution using the most common land use in each cell (the modal class):
reproject(  # You could also do this by applying the row_difference and cell_reduce method from the DEM.
    source=reclassified_data, destination=resampled_raster, src_transform=merged_transform,
    src_crs="EPSG:27700", dst_transform=new_transform, dst_crs="EPSG:27700",
    resampling=Resampling.mode  # Use the mode to get the value that is most common
)

# Change np.nan's back into -9999s:
resampled_raster[np.isnan(resampled_raster)] = -9999

# Write as an asc file:
output_path = f'{root}/Processed Data/UK Land Use {resolution_output}m'

# Save to file
with rasterio.open(
        output_path+'.asc', "w", driver="AAIGrid", height=resampled_raster.shape[0], width=resampled_raster.shape[1],
        count=1, dtype=resampled_raster.dtype, crs="EPSG:27700", transform=new_transform, nodata=-9999) as dst:
    dst.write(resampled_raster, 1)

## Lake Map

Use the soil data grid to build a raster that masks out the lakes. In previous versions, OS data was used to define lakes; however, when reviewing this it appears that this method is over zealous.

The lake map is used to alter the river parameters  in cells that contain lakes. Any river channel that is next to a lake cell have the Strickler overland flow reduced from either 20 to 3 or 50 to 10 (depending on which SHETRAN version is being used).

This is created using the CEH land use data but with a lower threshold for lake presence than the land use raster created above. This is because using the *mode* to describe the presence or absence of a lake tends to underestimate. Instead, a numpy array of 0's is built, water cells are added to it as 1's, then in the regridding process the average of the input cells is taken. This can then be compared to a threshold and cells >= the threshold set to lakes (1) and all other cells set to -9999.

A threshold of around 25% creates a lake map that (at 1km) matches relatively well to reality.

*TODO: Consider cross referencing this with OS lake data to remove rivers from the lakes dataset.*

In [ ]:
# Create an empty array to hold the reclassified data:
reclassified_lake_data = np.zeros(shape=merged_raster.shape)  # np.copy(merged_raster)

# Change -9999s into nan values so that they do not influence processing:
reclassified_lake_data[merged_raster == 15] = 1
reclassified_lake_data[merged_raster == 16] = 1

# Set up an empty array to hole the resampled data:
xmin, ymin, xmax, ymax = 0, 0, 661000, 1241000  # resolution of the existing SHETRAN inputs
new_transform = from_bounds(xmin, ymin, xmax, ymax,
                            width=(xmax - xmin) // resolution_output,
                            height=(ymax - ymin) // resolution_output)
new_shape = ((ymax - ymin) // resolution_output, (xmax - xmin) // resolution_output)
resampled_lake_raster = np.empty(new_shape)

# Resample the data to the desired resolution using the average value in each cell:
reproject(  # You could also do this by applying the row_difference and cell_reduce method from the DEM.
    source=reclassified_lake_data, destination=resampled_lake_raster, src_transform=merged_transform,
    src_crs="EPSG:27700", dst_transform=new_transform, dst_crs="EPSG:27700",
    resampling=Resampling.average  # Average value (0s & 1s)
)

# Convert cells that are over the lake threshold to 1s those below to -9999:
lake_threshold = 0.25  # If the threshold is over 0.5 then you can use Resampling.mode above instead
resampled_lake_raster[resampled_lake_raster>=lake_threshold] = 1
resampled_lake_raster[resampled_lake_raster<lake_threshold] = -9999

# Write as an asc file:
output_path = f'{root}/Processed Data/UK_Lake_Mask_{resolution_output}m'

# Save to file
with rasterio.open(
        output_path+'.asc', "w", driver="AAIGrid", height=resampled_lake_raster.shape[0], width=resampled_lake_raster.shape[1],
        count=1, dtype=resampled_lake_raster.dtype, crs="EPSG:27700", transform=new_transform, nodata=-9999) as dst:
    dst.write(resampled_lake_raster, 1)

## Vegetation Details

The vegetation details csv holds the default parameters for the different land use classes. These are displayed below:

| Veg Type # | Vegetation Type | Canopy storage capacity (mm) | Leaf area index | Maximum rooting depth (m) | AE/PE at field capacity | Strickler overland flow coefficient |
|------------|-----------------|-------------------------------|-----------------|----------------------------|-------------------------|-------------------------------------|
| 1          | Arable          | 1                             | 0.8             | 0.8                        | 0.6                     | 0.6                               |
| 2          | BareGround      | 0                             | 0               | 0.1                        | 0.4                     | 3                                 |
| 3          | Grass           | 1.5                           | 1               | 1                          | 0.6                     | 0.5                               |
| 4          | DeciduousForest | 5                             | 1               | 1.6                        | 1                       | 1                                 |
| 5          | EvergreenForest | 5                             | 1               | 2                          | 1                       | 0.25                              |
| 6          | Shrub           | 1.5                           | 1               | 1                          | 0.4                     | 2                                 |
| 7          | Urban           | 0.3                           | 0.3             | 0.5                        | 0.4                     | 5                                 |
| 8          | Water           | 0                             | 0               | 0.1                        | 0.4                     | 3                                 |
| 11         | UDM_MM2017_Rural| 3                             | 1               | 1                          | 0.63                    | 2                                 |
| 12         | UDM_MM2017_Urban_CombinedSewers| 0.3           | 0.3             | 0.5                        | 1                       | 12                                |
| 13         | UDM_Future_Urban_SeperatedSewers| 0.3          | 0.3             | 0.5                        | 1                       | 12                                |



In [4]:
# Create a dataframe of the above data:
df = pd.DataFrame([
    ['Veg Type #', 'Vegetation Type', 'Canopy storage capacity (mm)', 'Leaf area index',
     'Maximum rooting depth (m)', 'AE/PE at field capacity', 'Strickler overland flow coefficient'],
    [1, 'Arable', 1, 0.8, 0.8, 0.6, 0.6],
    [2, 'BareGround', 0, 0, 0.1, 0.4, 3],
    [3, 'Grass', 1.5, 1, 1, 0.6, 0.5],
    [4, 'DeciduousForest', 5, 1, 1.6, 1, 1],
    [5, 'EvergreenForest', 5, 1, 2, 1, 0.25],
    [6, 'Shrub', 1.5, 1, 1, 0.4, 2],
    [7, 'Urban', 0.3, 0.3, 0.5, 0.4, 5],
    [8, 'Water', 0, 0, 0.1, 0.4, 3],
    [11, 'UDM_MM2017_Rural', 3, 1, 1, 0.63, 2],
    [12, 'UDM_MM2017_Urban_CombinedSewers', 0.3, 0.3, 0.5, 1, 12],
    [13, 'UDM_Future_Urban_SeperatedSewers', 0.3, 0.3, 0.5, 1, 12],
])

# Write the DataFrame to a CSV file
csv_file = f'{root}/Processed Data/Vegetation_Details.csv'
df.to_csv(csv_file, index=False, header=False)

Data has been written to I:/SHETRAN_GB_2021/02_Input_Data/National Data Inputs for SHETRAN UK//Processed Data/vegetation_data.csv


## Superceeded LCM 2007 Code

The following code was used in testing, but not used in the final processing. Potentially due to running too slowly, such as with the vector LCM processing.

Processing steps:
1. Download data (manually, unzip if necessary).
2. Merge data classes as per the table below and save the updated shapefiles.
a. Load each regional shapefile in turn.
b. Remove the unnecessary data.
c. Dissolve the polygons to reduce the files size.
d. Write the shapefiles (these can be removed once the rest of these steps are completed).
3. Merge the shapefiles into a single UK wide vector dataset.
4. Read the UK dataset and resample into the desired resolution and write as asc files. This has the following steps:
a. Create a vector grid of the desired resolution covering the standard SHETRAN UK domain and give each cell an ID.
b. Intersect this grid with the UK land cover data so that each polygon is within a single cell boundary and has the grid ID that it is within.
c. Calculate the area of each intersected polygon, filter using the area and grid cell ID, and remove duplicates, leaving only a single polygon per grid cell (the one with the largest area).
d. Join these polygons back to the original grid (so that the data can be displayed as a regular grid, rather than 1 irregular polygon per grid cell).
e. Rasterise and save the data

In [29]:
# # >>> STEP 2 <<<
# # Define the reclassification dictionary
# reclass_dict = {  # (CEH LCM to SHETRAN Classes)
#     1: 4, 2: 5, 3: 1,
#     4: 3, 5: 3, 6: 3, 7: 3,8: 3,
#     9: 6, 10: 6, 11: 6, 12: 6, 13: 6,
#     14: 2, 15: 2, 16: 2,  17: 2,  18: 2,  19: 2, 20: 2, 21: 2,
#     22: 7, 23: 7
# }
#
# # List the shapefiles in GB:
# GB_LCM = os.path.join(root, 'Land Use Inputs/LCM_2007_vector_GB_Digimap/lcm-2007-vec_5779248')
# GB_LCM_files = os.listdir(GB_LCM)
# shapefiles = [os.path.join(GB_LCM, sf) for sf in GB_LCM_files if sf.endswith('.shp')]
#
# NI_LCM = os.path.join(root, 'Land Use Inputs/LCM_2007_vector_NI_Digimap/lcm-2007-vec-ni_4578539')
# NI_LCM_files = os.listdir(NI_LCM)
# shapefiles.append([os.path.join(NI_LCM, sf) for sf in NI_LCM_files if sf.endswith('.shp')])
#
# # Run through the files (including NI):
# counter = 1
# for shapefile in shapefiles:
#     print(counter, '/', len(shapefiles))
#
#     # Read in the data:
#     sf = gpd.read_file(shapefile)
#
#     # Reclassify from LCM to SHETRAN classes'
#     sf['SHETRAN'] = sf['INTCODE'].map(reclass_dict)
#
#     # Reproject the Northern Ireland file into BNG (from ING):
#     if 'LCM_2007_vector_NI_Digimap' in shapefile:
#         sf = sf.to_crs(epsg=27700)
#
#     # Cull the columns you don't need:
#     columns = sf.columns
#     columns = [column for column in columns if column not in ['SHETRAN', 'geometry']]
#     sf.drop(columns, inplace=True, axis=1)
#
#     # Dissolve the polygons to reduce file size:
#     sf_dissolved = sf.dissolve('SHETRAN')
#
#     # Save the updated shapefile:
#     sf_dissolved.to_file(
#         os.path.join(root, "Land Use Inputs/Reclassified shapefiles", os.path.basename(shapefile))
#     )
#
#     counter += 1

In [ ]:
# The projection files again don't quite match so I have manually copied the projection from the GB files to the NI file... This is a very small difference and does not seem to make any difference to the polygon locations.
#
# Original: PARAMETER["Scale_Factor",0.9996012717]
#
# Updated: PARAMETER["Scale_Factor",0.999601272]

# # >>> Step 3 <<<
# # List the shapefiles in GB:
# shapefile_path = os.path.join(root, 'Land Use Inputs/Reclassified shapefiles')
# shapefiles = os.listdir(shapefile_path)
# shapefiles = [os.path.join(shapefile_path, sf) for sf in shapefiles if sf.endswith('.shp')]
#
# # Merge into a single file:
# gdfs = []
# for shapefile in shapefiles:
#     gdfs.append(gpd.read_file(shapefile))
#
# # Merge all GeoDataFrames into one
# merged_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
#
# # Save the merged GeoDataFrame to a new shapefile
# merged_gdf.to_file(shapefile_path + '/LCM_2007_vector_UK_BNG.shp')

In [ ]:
# >>> Step 4 <<<
# The following code was used to process the vector LCM straight into raster form. I believe that it works, but is extremely slow (and did not finish when applied nationally."""

# from rasterio.merge import merge
# from rasterio.transform import from_bounds
#
# # Paths for your rasters
# raster_GB_LCM = root + "/Land Use Inputs/LCM 2007 25m Raster/data/lcm2007gb25m.tif"
# raster_NI_LCM = NI_LCM_BNG
#
# # Open LCM GB
# with rasterio.open(raster_GB_LCM) as raster:
#     raster_GB_array = raster.read(1)
#     transform_GB = raster.transform
#     meta_GB = raster.meta.copy()
#
# # Open LCM NI (in BNG)
# with rasterio.open(raster_GB_LCM) as raster:
#     raster_NI_array = raster.read(1)
#     transform_NI = raster.transform
#     meta_NI = raster.meta.copy()
#
# # Merge the two rasters
# merged_raster, merged_transform = merge([(raster_NI_array, transform_NI), (raster_GB_array, transform_GB)])
#
# # Regrid to 50m resolution
# xmin, ymin, xmax, ymax = 0, 0, 661000, 1241000  # Specified extent
# new_transform = from_bounds(xmin, ymin, xmax, ymax, width=(xmax - xmin) // 50, height=(ymax - ymin) // 50)
#
# new_shape = ((ymax - ymin) // 50, (xmax - xmin) // 50)
# resampled_raster = np.empty(new_shape, dtype=merged_raster.dtype)
#
# reproject(
#     source=merged_raster,
#     destination=resampled_raster,
#     src_transform=merged_transform,
#     src_crs="EPSG:27700",
#     dst_transform=new_transform,
#     dst_crs="EPSG:27700",
#     resampling=Resampling.mode  # Or Resampling.average for continuous data
# )
#
# # Save to file
# output_path = "merged_resampled_50m.tif"
# with rasterio.open(
#     output_path,
#     "w",
#     driver="GTiff",
#     height=resampled_raster.shape[0],
#     width=resampled_raster.shape[1],
#     count=1,
#     dtype=resampled_raster.dtype,
#     crs="EPSG:27700",
#     transform=new_transform
# ) as dst:
#     dst.write(resampled_raster, 1)
#
# print(f"Resampled raster saved to {output_path}")


In [ ]:
# # >>> Step 4 <<<
# # Load the vector data (merged shapefile)
# gdf = gpd.read_file(shapefile_path + '/LCM_2007_vector_UK_BNG.shp')
# xmin, ymin, xmax, ymax = 0, 0, 661000, 1241000  # British National Grid boundaries
#
# # TEST DATASET
# # gdf = gpd.read_file(shapefile_path + '/nk_land_parcel.shp')
# # xmin, ymin, xmax, ymax = 399500, 822000, 414500, 868000
#
# # Step 2: Create a vector grid
# cell_size = resolution_output  # 100m resolution
# cols = np.arange(xmin, xmax, cell_size)
# rows = np.arange(ymin, ymax, cell_size)
#
# grid_cells = []
# for x in cols:
#     for y in rows:
#         grid_cells.append(box(x, y, x + cell_size, y + cell_size))
#
# # Turn this into a geodataframe and give it an ID
# grid = gpd.GeoDataFrame({"geometry": grid_cells}, crs=gdf.crs)
# grid['ID'] = np.arange(0, grid.shape[0])
#
# # Step 1: Intersect the grid and the shapefile
# intersected = gpd.overlay(grid, gdf, how='intersection', keep_geom_type=False)
#
# # Step 2: Calculate the area of each intersected polygon
# intersected["area"] = intersected.area
#
# # Step 3: Sort the intersected DataFrame by 'ID' and 'area' and crop to only the largest land type per cell:
# intersected_sorted = intersected.sort_values(by=["ID", "area"], ascending=[True, False])
#
# # Step 4: Drop duplicates based on 'ID', keeping only the largest area
# filtered_intersected = intersected_sorted.drop_duplicates(subset="ID")
# # filtered_intersected.to_file(shapefile_path + '/filtered_intersected.shp')
#
# # 5. Converting filtered_intersected straight to raster misses cells, instead join the LC classes back to the grid:
# # Perform the left join on the 'ID' column
# grid_with_intersected = grid.merge(filtered_intersected[['SHETRAN', 'ID']], on="ID", how="left", suffixes=('_grid', '_intersected'))
# # grid_with_intersected.to_file(shapefile_path + '/grid_with_intersected.shp')
#
# # Step 6: Rasterize the intersected polygons
# # Define the raster properties
# transform = rasterio.transform.from_bounds(xmin, ymin, xmax, ymax, len(cols), len(rows))
#
# # Prepare shapes and values for rasterisation:
# shapes = ((geom, value) for geom, value in zip(grid_with_intersected.geometry, grid_with_intersected['SHETRAN']))
#
# # Rasterize:
# raster = rasterio.features.rasterize(
#     shapes,
#     out_shape=(len(rows), len(cols)),
#     transform=transform,
#     fill=-9999,  # NoData value
#     dtype="int32"
# )
#
# # Convert 0s to -9999s for no data values:
# raster[raster == 0] = -9999
#
# write_ascii(
#     array=raster,
#     ascii_ouput_path=f'{root}/Processed Data/CEH_LCM_2007 {resolution_output}m.asc',
#     xllcorner=xmin,
#     yllcorner=ymin,
#     cellsize=cell_size,
#     NODATA_value=-9999,
#     data_format='%1.0f'
# )